In [17]:
import pandas as pd
df = pd.read_csv(r'Transactions.csv')
print(df.dtypes)
# Avax Fiat Buys
avax_buy = df[(df["Type"] == "Fiat Buy") & (df["Asset"] == "AVAX")]
# Avax Fiat Buy Cost Basis
avax_buy_usd = df[(df['Internal Id'].isin(avax_buy["Internal Id"])) 
                  & (df['Asset'] == 'USD') 
                  & (df['Record Type'] == 'Debit')]

no_of_avax_tokens = avax_buy['Amount'].to_list()
cost_of_avax_tokens = avax_buy_usd['Amount'].to_list()
cost_basis_per_token = [-cost/no for (cost, no) in zip(cost_of_avax_tokens, no_of_avax_tokens)]
#print(cost_basis_per_token)
assert len(no_of_avax_tokens) == len(cost_of_avax_tokens) == len(cost_basis_per_token)

avax_token_cost_hifo_list = [(no,cost) for cost, no in sorted(zip(cost_basis_per_token, no_of_avax_tokens), reverse=True)]

# Avax Fiat Sells
avax_sell = -df[((df["Type"] == "Fiat Sell") | (df["Type"] == "Trade")) & (df["Asset"] == "AVAX")]['Amount']
avax_sell = avax_sell.to_list()
avax_sell_i = 0
avax_hifo_i = 0

while avax_sell_i < len(avax_sell):
    cur_sell_avax_tokens = avax_sell[avax_sell_i]
    cur_sell_avax_tokens_copy = cur_sell_avax_tokens
    cur_buy_avax_tokens, cur_price = avax_token_cost_hifo_list[avax_hifo_i] 

    cur_min = min(cur_sell_avax_tokens, cur_buy_avax_tokens)

    cur_sell_avax_tokens -= cur_min
    cur_buy_avax_tokens -= cur_min

    if cur_sell_avax_tokens == 0:
        avax_sell_i += 1
    if cur_buy_avax_tokens == 0:
        avax_hifo_i += 1
    
    # Record Cost Basis Transaction
    print(cur_sell_avax_tokens_copy, cur_min, cur_price)


Timestamp (UTC)     object
Type                object
Internal Id          int64
Platform            object
Platform Id        float64
Blockchain Id       object
Record Type         object
Asset               object
Amount             float64
Description        float64
dtype: object
0.0 0.08 128.98
0.0 0.92 128.98
0.0 1.0 128.98
6.42100000011413 5.0 89.25
7.89000000011413 3.531 88.51
10.05400000011413 1.367 88.51
6.31900000011413 5.102 88.50999999999999
1.4210000001141303 10.0 87.0
6.42100000011413 5.0 87.0
2.1100000001141304 9.311 86.35000000000001
9.221000000114131 2.2 86.34
0.0 11.42100000011413 86.31
0.0 11.42897101900432 86.31
[0.08, 0.92, 1.0, 11.42100000011413, 11.42897101900432]
